In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.text import *
import numpy as np

In [3]:
path = Path('data')
path

PosixPath('data')

### Load test data

In [4]:
test_df = pd.read_csv(path/'test.csv')
test_df.head()

,itemid,title,image_path
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg
1,637234604,maybelline clear smooth all in one bb cream sp...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg
2,690282890,murah innisfree eco natural green tea bb cream...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg
3,930913462,loreal white perfect day cream spf 17 pa white...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg
4,1039280071,hada labo cc cream ultimate anti aging spf 35 ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg


adding category column

In [5]:
test_df['Category'] = 0
test_df.head()

,itemid,title,image_path,Category
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg,0
1,637234604,maybelline clear smooth all in one bb cream sp...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg,0
2,690282890,murah innisfree eco natural green tea bb cream...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg,0
3,930913462,loreal white perfect day cream spf 17 pa white...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg,0
4,1039280071,hada labo cc cream ultimate anti aging spf 35 ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg,0


adding big category to titles

In [6]:
for index, row in test_df.iterrows():
    big_category = row['image_path'].split('/')[0].split('_')[0]
    test_df.iat[index, 1] = big_category + ' ' + row['title']
test_df.head()

,itemid,title,image_path,Category
0,370855998,beauty flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg,0
1,637234604,beauty maybelline clear smooth all in one bb c...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg,0
2,690282890,beauty murah innisfree eco natural green tea b...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg,0
3,930913462,beauty loreal white perfect day cream spf 17 p...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg,0
4,1039280071,beauty hada labo cc cream ultimate anti aging ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg,0


### Load NLP model

In [7]:
learn_nlp = load_learner(path, fname = 'export_nlp_pref.pkl')

### Load image model

In [8]:
learn_image = load_learner(path, fname = 'export.pkl')

### Run predictions on every row

In [9]:
image_path = path/'images'
for index, row in test_df.iterrows():
    pred_nlp = learn_nlp.predict(row['title'])[2];
    pred_img = learn_image.predict(open_image(image_path/row['image_path']))[2]
    pred_sum = pred_nlp.add(pred_img)
    test_df.iat[index, 3] = np.argmax(pred_sum)
test_df.head()

,itemid,title,image_path,Category
0,370855998,beauty flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg,5
1,637234604,beauty maybelline clear smooth all in one bb c...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg,5
2,690282890,beauty murah innisfree eco natural green tea b...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg,5
3,930913462,beauty loreal white perfect day cream spf 17 p...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg,4
4,1039280071,beauty hada labo cc cream ultimate anti aging ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg,5


In [10]:
df = pd.DataFrame({'itemid':test_df['itemid'], 'Category':test_df['Category']}, columns=['itemid', 'Category'])

In [11]:
df.to_csv(path/'results_nlp_img.csv', index=False)